In [1]:
import time
import os
import sys

# if os.environ.get('ROS_DISTRO'):
#     import rospy
import pybullet as p

from cairo_simulator.core.utils import ASSETS_PATH
from cairo_simulator.core.log import Logger
from cairo_simulator.core.simulator import Simulator, SimObject
from cairo_simulator.devices.manipulators import Sawyer
from utils import load_configuration, save_config_to_configuration_file, manual_control, create_cuboid_obstacle


def main():
#     if os.environ.get('ROS_DISTRO'):
#         rospy.init_node("CAIRO_Sawyer_Simulator")
#         use_ros = True
#     else:
#         use_ros = False
    use_ros = False
    use_real_time = True
    logger = Logger()
    sim = Simulator(logger=logger, use_ros=use_ros, use_real_time=use_real_time) # Initialize the Simulator
    p.setGravity(0,0,-9.81)
    p.configureDebugVisualizer(p.COV_ENABLE_RENDERING,0)
    p.setPhysicsEngineParameter(enableFileCaching=0)
    ground_plane = SimObject("Ground", "plane.urdf", [0,0,0])
    # Add a table and a Sawyer robot
    #table = SimObject("Table", ASSETS_PATH + 'table.sdf', (0.9, 0, 0), (0, 0, 1.5708),fixed_base=1) #rotated 90deg (in radians)along z-axis
    
    table = SimObject('table', 'NEWtable.urdf',  (0.9, 0.1, .47),(1.5708*2,0,0),fixed_base=1)
    #table = SimObject('table', 'NEWtable.urdf',  (3, 0.1, .47),(1.5708*2,0,0),fixed_base=0)

    print(p.getNumJoints(table.get_simulator_id()))
    sawyer_robot = Sawyer("sawyer0", [0, 0, 0.8], fixed_base=1)
    
    p.configureDebugVisualizer(p.COV_ENABLE_RENDERING,1)
    
    # sim_obj1 = SimObject('cube0', 'cube_small.urdf', (0.75, 0, .55)) 
    
    block = SimObject('block1', '1block.urdf', (0.69, -0.2, .55))
#     sim_obj1 = SimObject('hole1', '1.5hole.urdf',  (0.69, 0.1, .550),(0,0,1.5708),fixed_base=1)
#     sim_obj2 = SimObject('hole2', '1.25hole.urdf', (0.69, 0.3, .550),(0,0,1.5708),fixed_base=1)
#     sim_obj3 = SimObject('hole3', '1.15hole.urdf', (0.69, 0.5, .550),(0,0,1.5708),fixed_base=1)

    sim_obj1 = SimObject('hole1', '1.5hole.urdf',  (0.69, 0.1, .530),(0,0,1.5708),fixed_base=1)
    sim_obj2 = SimObject('hole2', '1.25hole.urdf', (0.69, 0.3, .530),(0,0,1.5708),fixed_base=1)
    sim_obj3 = SimObject('hole3', '1.15hole.urdf', (0.69, 0.5, .530),(0,0,1.5708),fixed_base=1)    
    
    #solve_inverse_kinematics(self, target_position, target_orientation=None, target_in_local_coords=False)
    
    #joint_config = sawyer_robot.solve_inverse_kinematics([0.9,0,1.5], [0,0,0,1])
    
    #xoffset=0.0635  #2.5in from corner of holeblocks to center
#     xoffset=0
#     #yoffset=-0.0635  #2.5in from corner of holeblocks to center
#     yoffset=0


    orientation= [0,1,0,0] #quaternion hand pointing down..ish
    
    
#     joint_config1 = [0.69+xoffset, 0.1+yoffset, 1.0]
#     joint_config2 = [0.69+xoffset, 0.1+yoffset, 0.8]
#     joint_config3 = [0.69+xoffset, 0.3+yoffset, 1.0]
#     joint_config4 = [0.69+xoffset, 0.3+yoffset, 0.8]
#     joint_config5 = [0.69+xoffset, 0.5+yoffset, 1.0]
#     joint_config6 = [0.69+xoffset, 0.5+yoffset, 0.8]
#     configlist=[joint_config1,joint_config2,joint_config3,joint_config4,joint_config5,joint_config6]
    
    """
    for x in range(len(configlist)):
        joint_config = sawyer_robot.solve_inverse_kinematics(configlist[x],orientation)
        sawyer_robot.move_to_joint_pos(joint_config)
        time.sleep(4)
    """
    
    #joint_config = sawyer_robot.solve_inverse_kinematics(joint_config2,orientation)
    #sawyer_robot.move_to_joint_pos(joint_config)
    xoffset=0
    yoffset=0
    
    joint_config2 = [0.69+xoffset, 0.1+yoffset, 1]
    joint_config = sawyer_robot.solve_inverse_kinematics(joint_config2,orientation)
    sawyer_robot.move_to_joint_pos(joint_config)
    time.sleep(1.)
    joint_config2 = [0.69+xoffset, 0.1+yoffset, 0.8]
    joint_config = sawyer_robot.solve_inverse_kinematics(joint_config2,orientation)
    sawyer_robot.move_to_joint_pos(joint_config)
    
    
    orig_key = ord('0') 
    z_inc_key = ord('1') 
    z_dec_key = ord('2') 
    y_inc_key = ord('3') 
    y_dec_key = ord('4') 
    
    a_inc_key = ord('5') #target 2 and 3
    a_dec_key = ord('6') 
    
    
    loop_count = 0
    # Loop until someone shuts us down
    h=0.75
    while True:
            #xoffset=0.0635  #2.5in from corner of holeblocks to center
            xoffset=0
            #yoffset=-0.0635  #2.5in from corner of holeblocks to center
            yoffset=0
            
            # Control the sensor position
            keys = p.getKeyboardEvents()
            if z_inc_key in keys and keys[z_inc_key] & p.KEY_WAS_TRIGGERED:
                xoffset=-0.0508
                yoffset=0
                joint_config2 = [0.69+xoffset, 0.1+yoffset, h]
                joint_config = sawyer_robot.solve_inverse_kinematics(joint_config2,orientation)
                sawyer_robot.move_to_joint_pos(joint_config)
            if z_dec_key in keys and keys[z_dec_key] & p.KEY_WAS_TRIGGERED:
                xoffset=0
                yoffset=0.0508
                joint_config2 = [0.69+xoffset, 0.1+yoffset, h]
                joint_config = sawyer_robot.solve_inverse_kinematics(joint_config2,orientation)
                sawyer_robot.move_to_joint_pos(joint_config)
            if y_inc_key in keys and keys[y_inc_key] & p.KEY_WAS_TRIGGERED:
                xoffset=-0.0508
                yoffset=0.0508
                joint_config2 = [0.69+xoffset, 0.1+yoffset, h]
                joint_config = sawyer_robot.solve_inverse_kinematics(joint_config2,orientation)
                sawyer_robot.move_to_joint_pos(joint_config)
            if y_dec_key in keys and keys[y_dec_key] & p.KEY_WAS_TRIGGERED:
                xoffset=-0.0508
                yoffset=-0.0508
                joint_config2 = [0.69+xoffset, 0.1+yoffset, h]
                joint_config = sawyer_robot.solve_inverse_kinematics(joint_config2,orientation)
                sawyer_robot.move_to_joint_pos(joint_config)
            if orig_key in keys and keys[orig_key] & p.KEY_WAS_TRIGGERED:
                xoffset=0
                yoffset=0    
                joint_config2 = [0.69, 0.1, h]
                joint_config = sawyer_robot.solve_inverse_kinematics(joint_config2,orientation)
                sawyer_robot.move_to_joint_pos(joint_config)
                
            if a_dec_key in keys and keys[a_dec_key] & p.KEY_WAS_TRIGGERED:
                xoffset=0
                yoffset=0
                joint_config2 = [0.69+xoffset, 0.3, h]
                joint_config = sawyer_robot.solve_inverse_kinematics(joint_config2,orientation)
                sawyer_robot.move_to_joint_pos(joint_config)
            if a_inc_key in keys and keys[a_inc_key] & p.KEY_WAS_TRIGGERED:
                xoffset=0
                yoffset=0    
                joint_config2 = [0.69+xoffset, 0.5, h]
                joint_config = sawyer_robot.solve_inverse_kinematics(joint_config2,orientation)
                sawyer_robot.move_to_joint_pos(joint_config)
 
            time.sleep(1.)
    
    #manual_control(sawyer_robot)    
    # Loop until someone shuts us down
    try:
        while True:
            sim.step()
    except KeyboardInterrupt:
        p.disconnect()
        sys.exit(0)

In [ ]:
main()


0


In [1]:

#from sensor.py
# Simple demo showing how to use a LaserRangeFinder sensor within CAIRO Sim.
# Use the 1-4 keys to move the sensor around, check the terminal for readings from the sensor.
import time
import os
import sys

if os.environ.get('ROS_DISTRO'):
    import rospy
import pybullet as p
import numpy as np
from cairo_simulator.core.simulator import Simulator, SimObject
from cairo_simulator.core.utils import ASSETS_PATH
from cairo_simulator.core.log import Logger
from cairo_simulator.devices.sensors import LaserRangeFinder

def main():
    if os.environ.get('ROS_DISTRO'):
        rospy.init_node("CAIRO_Simulator_LRFSensorTest")
        use_ros = True
    else:
        use_ros = False

    use_real_time = True

    logger = Logger()
    sim = Simulator(logger=logger, use_ros=use_ros, use_real_time=use_real_time) # Initialize the Simulator
    p.setGravity(0, 0, 0)
    p.configureDebugVisualizer(p.COV_ENABLE_RENDERING,0)
    p.setPhysicsEngineParameter(enableFileCaching=0)

    # Add a few 0.05m x 0.05m x 0.05m cubes
    for idx in range(1,10):
        cube = SimObject(object_name="Cube%d"%idx, model_file_or_sim_id="cube_small.urdf", position=[0,0,idx/4.], orientation=[0,0,0,1])
        cube.move_to_pose([0,0,idx/4.],[0,0,0,1])
    # Add the sensor
    lrf_sensor = LaserRangeFinder(position_offset=[-0.5,0,0.3], fixed_pose=False)
    lrf_sensor.set_range(0.3,1.0) # Sensor takes readings from 0.3m to 1.0m away from its position.
    lrf_sensor.set_debug_mode(True)

    p.configureDebugVisualizer(p.COV_ENABLE_RENDERING,1)
    z_inc_key = ord('1') # Press 1 to increase Z position of sensor
    z_dec_key = ord('2') # Press 2 to decrease Z position of sensor
    y_inc_key = ord('3') # Press 3 to increase Y position of sensor
    y_dec_key = ord('4') # Press 4 to decrease Y position of sensor

    loop_count = 0
    # Loop until someone shuts us down
    try:
        while True:

            # Control the sensor position
            keys = p.getKeyboardEvents()
            if z_inc_key in keys and keys[z_inc_key] & p.KEY_WAS_TRIGGERED:
                lrf_sensor._position_offset += np.array([0, 0, 0.05,])
            if z_dec_key in keys and keys[z_dec_key] & p.KEY_WAS_TRIGGERED:
                lrf_sensor._position_offset -= np.array([0, 0, 0.05,]) 
            if y_inc_key in keys and keys[y_inc_key] & p.KEY_WAS_TRIGGERED:
                lrf_sensor._position_offset += np.array([0, 0.05, 0,])
            if y_dec_key in keys and keys[y_dec_key] & p.KEY_WAS_TRIGGERED:
                lrf_sensor._position_offset -= np.array([0, 0.05, 0,]) 

            # Get and display a reading from the sensor
            if loop_count % 10 == 0:
                dist = lrf_sensor.get_reading()
                sim.logger.info("Detector Pos: %s \t\t\t Det: %f" % (lrf_sensor._position_offset, dist))
                loop_count = 0
            loop_count += 1            
            sim.step()
            time.sleep(1./60.)
    except KeyboardInterrupt:
        p.disconnect()
        sys.exit(0)
   

In [ ]:
main()

Detector Pos: [-0.5  0.   0.3] 			 Det: inf
Detector Pos: [-0.5  0.   0.3] 			 Det: inf
Detector Pos: [-0.5  0.   0.3] 			 Det: inf
Detector Pos: [-0.5  0.   0.3] 			 Det: inf
Detector Pos: [-0.5  0.   0.3] 			 Det: inf
Detector Pos: [-0.5  0.   0.3] 			 Det: inf
Detector Pos: [-0.5  0.   0.3] 			 Det: inf
Detector Pos: [-0.5  0.   0.3] 			 Det: inf
Detector Pos: [-0.5  0.   0.3] 			 Det: inf
Detector Pos: [-0.5  0.   0.3] 			 Det: inf
Detector Pos: [-0.5  0.   0.3] 			 Det: inf
Detector Pos: [-0.5  0.   0.3] 			 Det: inf
Detector Pos: [-0.5   0.    0.35] 			 Det: inf
Detector Pos: [-0.5   0.    0.35] 			 Det: inf
Detector Pos: [-0.5   0.    0.35] 			 Det: inf
Detector Pos: [-0.5  0.   0.3] 			 Det: inf
Detector Pos: [-0.5  0.   0.3] 			 Det: inf
Detector Pos: [-0.5  0.   0.3] 			 Det: inf
Detector Pos: [-0.5   0.05  0.3 ] 			 Det: inf
Detector Pos: [-0.5   0.05  0.3 ] 			 Det: inf
Detector Pos: [-0.5  0.1  0.3] 			 Det: inf
Detector Pos: [-0.5  0.1  0.3] 			 Det: inf
Detector Pos: [-0

In [1]:
#from test_manual_control.py
import time
import os
import sys
from pprint import pprint

import pybullet as p
import numpy as np

from cairo_simulator.core.utils import ASSETS_PATH
from cairo_simulator.core.log import Logger
from cairo_simulator.core.simulator import Simulator, SimObject
from cairo_simulator.devices.manipulators import Sawyer
from cairo_simulator.core.link import get_link_pairs, get_joint_info_by_name
from utils import load_configuration, save_config_to_configuration_file, manual_control, create_cuboid_obstacle

def init_sim_with_sawyer():
    # Setup simulator
    use_ros = False
    use_real_time = True
    logger = Logger()
    sim = Simulator(logger=logger, use_ros=use_ros, use_real_time=use_real_time)  # Initialize the Simulator
    p.configureDebugVisualizer(p.COV_ENABLE_RENDERING, 0)
    p.setPhysicsEngineParameter(enableFileCaching=0)
    ground_plane = SimObject("Ground", "plane.urdf", [0, 0, 0])
    # Add a table and a Sawyer robot
    print("Path = ", os.path.abspath(ASSETS_PATH))
    table = SimObject("Table", ASSETS_PATH + 'table.sdf', (0.9, 0, 0),
                      (0, 0, 1.5708))  # Table rotated 90deg along z-axis
    sawyer_robot = Sawyer("sawyer0", [0, 0, 0.8], fixed_base=1)

    p.configureDebugVisualizer(p.COV_ENABLE_RENDERING, 1)

    # Add small cubes on the table (table top bounds are known in world coordinate)
    # sim_obj1 = SimObject('cube0', 'cube_small.urdf', (0.75, 0, .55))
    # sim_obj2 = SimObject('cube1', 'cube_small.urdf', (0.74, 0.05, .55))
    # sim_obj3 = SimObject('cube2', 'cube_small.urdf', (0.67, -0.1, .55))
    # sim_obj4 = SimObject('cube3', 'cube_small.urdf', (0.69, 0.1, .55))

    sim_obj1 = create_cuboid_obstacle(name='box0', shape=p.GEOM_BOX, mass=1, position=[0.74, 0.05, .55],
                                      size=[0.09, 0.09, 0.35])
    # Interested in only PyBullet object IDs for obstacles
    obstacles = [sim_obj1, table.get_simulator_id()]

    return sim, sawyer_robot, obstacles

def main():
    # Load sawyer specific configuration
    sawyer_configuration = load_configuration('sawyer_configuration.json')
    sample_configurations = sawyer_configuration['sample_configurations']

    # Setup simulator
    sim, sawyer_robot, obstacles = init_sim_with_sawyer()

    # Moving Sawyer to a start position
    sawyer_robot.move_to_joint_pos(sample_configurations['config0'])
    time.sleep(2)
    manual_control(sawyer_robot)



In [ ]:
main()

Path =  /home/scott/ScheragaThesis/cairo_simulator/assets
